In [ ]:
# List of modules that are not installed in the course
!pip install OpenPermID
!pip install geocoder

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Downloading all bonds ever owned in CSPP

In [ ]:
from functions import downloadECBBonds

In [ ]:
holdingsECB = downloadECBBonds.download_ECB_Bonds()

# ECB green bonds that are listed on Euronext stock exchange 

TODO: make 1 function in .py file

In [ ]:
#reading data of green bonds listed on Euronext 
euronext_greenbonds = pd.read_excel("data/Euronext-Green-Bond-List.xlsx", header=0)
euronext_greenbonds

In [ ]:
#Getting the ISIN of the Euronext green bonds 
euronext_greenbond_isin = euronext_greenbonds["ISIN"]
euronext_greenbond_isin

In [ ]:
#Comparing the ISINs of the ECB and Euronext green bonds and returning matches 
ecbgreenbonds = holdingsECB[(holdingsECB["ISIN"].isin(euronext_greenbond_isin))]
ecbgreenbonds

TODO: calculate percentage green before and after 2020

# Read all the Eikon data

In [ ]:
from functions import readEikonData
eikon_data_complete = readEikonData.get_eikon_data_complete()
eligible_complete = readEikonData.get_data_eligible_complete() # read all data from eligible universe

Compare how many bonds from the eligible universe the ECB bought.

In [ ]:
overlap = eikon_data_complete[(eikon_data_complete["ISIN"].isin(eligible_complete["ISIN"]))]
print("the percentage of bonds bought by ECB in eligible universe that we have info on:    ", 
      overlap.shape[0]/eligible_complete.shape[0])

__More companies mapped in Eikon database (370 vs 339), but more industries mapped in PermID database (313 vs 158).__
TODO: Move this conclusion from the comparison between Eikon and PermID to a better place.

Read the dates

In [ ]:
years_issuer_bought = readEikonData.get_dates_data_frame()

# Sector Analysis

## Get all sector data from PermID

In [ ]:
from functions import permidSectors
sector_mappings = permidSectors.get_sector_mappings(holdingsECB)
sector_mappings.count()

## Represent industries and sectors

Create a pie chart to represent the number of times an industry, sector or supersector is present in the data (multiple bonds for the same company are counted separately)

In [ ]:
from functions import pieCharts

In [ ]:
pieCharts.make_pie_chart('hasPrimaryBusinessSector', sector_mappings)

In [ ]:
pieCharts.make_pie_chart('hasPrimaryEconomicSector', sector_mappings)

In [ ]:
# make_pie_chart('hasPrimaryIndustryGroup') # TODO: Fix Error

## Which industries and sectors are green?

Map the dictionary with greenness of all PermID sectors with the ones in which ECB invested



In [ ]:
from functions import sectors
sector_green_dict = sectors.get_sector_green_dict()

In [ ]:
primary_business_sector = pieCharts.get_all_sectors("hasPrimaryBusinessSector", sector_mappings)

primary_business_sector = sectors.map_green_dict_to_data_frame(primary_business_sector)
print(primary_business_sector)

primary_economic_sector = pieCharts.get_all_sectors("hasPrimaryEconomicSector", sector_mappings)
primary_economic_sector = sectors.map_green_dict_to_data_frame(primary_economic_sector)
print(primary_economic_sector)

TODO:
- Total Count for each sector overall and compare these sectors (we view as “bad”, like cement, oil&gas or with kaggle dataset(emission))
- Similar approach but per year → do we see a change in habit, total counts → differences in the sectors 
- For the sectors: make spaghetti plot with for each sector the percentage invested in every year (in app: select which sectors you want)
    - Include in app (can view all, and have “buttons” to choose what to show) 

# Environmental Analysis

## CO2 Data/Graphs

(Maarten): I think everything here can be deleted, as we're not able to make a comparison between the holdings and the eligible universe

### Spaghetti plot CO$_2$

In [ ]:
from functions import CO2
CO2.make_CO2_spaghetti_plot()

### Histogram change in CO$_2$

In [ ]:
CO2.make_CO2_histogram()

### emission of 2021 as a function of the emission in 2015

In [ ]:
CO2.plot_2020_2015()

Make a linear fit through for this graph (see github of course: data analytics > radient_descent)

In [ ]:
CO2.make_linear_fit()

As the slope is slightly below one, we can conclude that in general, the normalized CO$_2$ emission decreases.

## ESG data/plots

Function for cleaning the ESG data

In [ ]:
from functions import ESG
esg_company_data_eligible = ESG.cleaning_esg_data(eligible_complete)
esg_company_data_holdings = ESG.cleaning_esg_data(eikon_data_complete)
ESG.plot_ESG(esg_company_data_holdings)

### Spaghetti plot ESG Scores

### Some Descriptive Statistics

In [ ]:
esg_company_data_holdings.describe()

In [ ]:
esg_company_data_eligible.describe()

In [ ]:
ESG.plot_ESG_error_bars(esg_company_data_eligible, esg_company_data_holdings)

In this figure, we can clearly see that the average ESG-score of the companies increases.

In [ ]:
esg_company_data_holdings.iloc[:,1:].boxplot(figsize=(10,10),color='y')
esg_company_data_eligible.iloc[:,1:].boxplot(figsize=(10,10),color='r')

### ESG evolution of each company

fit the evolution of each company

Make a boxplot of the coefficients of these fits. This should give an indication about the general evolution (increase vs decrease).

In [ ]:
ESG.coefficients_box_plot(esg_company_data_eligible, esg_company_data_holdings)

This figure shows us that only roughly 25% of companies have a negative slope, while the other 75% have an increasin ESG score.

Let us now try to find a relation between the initial ESG score and the most recent one. This can be done by plotting and calculating the covariance matrix.

In [ ]:
ESG.covariance_matrix_plot(esg_company_data_eligible, esg_company_data_holdings)

In [ ]:
ESG.get_correlation_coefficients(esg_company_data_eligible, esg_company_data_holdings)

There seems to be some negative correlation ==> lower initial value, faster increase in ESG score.

### ESG Evolution of holdings through the years

In [ ]:
esg_scores_per_year = ESG.get_ESG_scores_per_year(years_issuer_bought, esg_company_data_holdings) # TODO: remove line if not used in notebook

In [ ]:
ESG.average_company_esg_score_plot(years_issuer_bought, esg_company_data_holdings)